## Load Model from .pt file

In [2]:
import torch

typeformer_state = torch.load('TypeFormer_pretrained.pt', map_location='cpu', weights_only=True)
altered_state = { key.replace('.net.3', '.net.2').replace('net','ff'): value for key, value in typeformer_state.items()}
altered_state

{'transformer_pre.model.layers.0.self_attn.linears.0.weight': tensor([[ 0.1929, -0.1434,  0.1674,  ..., -0.2686, -0.0792, -0.2879],
         [-0.5098, -0.0883,  0.1339,  ...,  0.2154,  0.0339, -0.2529],
         [ 0.0295, -0.1906, -0.0567,  ..., -0.1459, -0.1303,  0.1134],
         ...,
         [-0.3129, -0.0735, -0.1781,  ...,  0.1442, -0.2938,  0.3018],
         [ 0.1674, -0.0591,  0.1731,  ...,  0.2861, -0.0229,  0.0705],
         [ 0.4894,  0.2884,  0.3042,  ...,  0.1915,  0.1011, -0.1972]],
        dtype=torch.float64),
 'transformer_pre.model.layers.0.self_attn.linears.0.bias': tensor([ 0.1978, -0.0811,  0.4387,  0.2889, -0.2096, -0.3072,  0.0796, -0.3571,
         -0.3882,  0.1370,  0.4042, -0.7931, -0.1303, -0.7332, -0.7641, -0.1119,
          0.3365,  0.3808, -0.4955, -0.4895, -0.2097, -0.3209, -0.3213, -0.0183,
         -0.1759,  0.3475, -0.2716,  0.0756,  0.3113,  0.5932, -0.0885,  0.0647,
         -0.3849, -0.2654,  0.1210, -0.5482, -0.5969,  0.2672,  0.5791, -0.2713,
    

In [3]:
x = torch.tensor([[12., 4., 55., 11., 0.2]])
x

tensor([[12.0000,  4.0000, 55.0000, 11.0000,  0.2000]])

In [4]:
from model.Model import HARTrans
from config.train_config import configs
transformer = HARTrans(configs).double()
[ item for item in transformer.state_dict().keys() if '_rec' in item]

['transformer_rec.model.layers.0.self_attn.rotary_pos_emb.inv_freq',
 'transformer_rec.model.layers.0.self_attn.input_self_attn.norm.g',
 'transformer_rec.model.layers.0.self_attn.input_self_attn.to_q.weight',
 'transformer_rec.model.layers.0.self_attn.input_self_attn.to_kv.weight',
 'transformer_rec.model.layers.0.self_attn.input_self_attn.to_out.weight',
 'transformer_rec.model.layers.0.self_attn.input_self_attn.to_out.bias',
 'transformer_rec.model.layers.0.self_attn.state_self_attn.norm.g',
 'transformer_rec.model.layers.0.self_attn.state_self_attn.to_q.weight',
 'transformer_rec.model.layers.0.self_attn.state_self_attn.to_kv.weight',
 'transformer_rec.model.layers.0.self_attn.state_self_attn.to_out.weight',
 'transformer_rec.model.layers.0.self_attn.state_self_attn.to_out.bias',
 'transformer_rec.model.layers.0.self_attn.input_state_cross_attn.norm.g',
 'transformer_rec.model.layers.0.self_attn.input_state_cross_attn.to_q.weight',
 'transformer_rec.model.layers.0.self_attn.input_s

In [5]:
transformer.load_state_dict(altered_state, strict=True)

<All keys matched successfully>

In [6]:
transformer(x)

tensor([[0.9513, 0.0138, 0.0148, 0.8359, 0.7919, 0.8066, 0.0957, 0.0011, 0.8679,
         0.4171, 0.0675, 0.3885, 0.8291, 0.1909, 0.1047, 0.3542, 0.3868, 0.2232,
         0.4034, 0.1064, 0.7260, 0.1088, 0.0586, 0.3152, 0.2504, 0.8480, 0.2091,
         0.9862, 0.0525, 0.1353, 0.8392, 0.8372, 0.9021, 0.2002, 0.8880, 0.0745,
         0.7025, 0.2850, 0.8415, 0.8161, 0.1038, 0.0674, 0.1868, 0.1631, 0.7519,
         0.8224, 0.1597, 0.6773, 0.1259, 0.3615, 0.9181, 0.8282, 0.8740, 0.3951,
         0.0941, 0.0879, 0.8221, 0.1499, 0.7228, 0.7236, 0.7637, 0.7540, 0.0949,
         0.1022]], dtype=torch.float64, grad_fn=<SigmoidBackward0>)

## Create input from time-series event of keystrokes

In [7]:
import pandas as pd
import numpy as np

df = pd.read_csv('../keystrokes_4.csv')
df['timestamp'] = df['timestamp'].map(lambda x: x/1000_000)

In [8]:
df.head()

,event_type,key,timestamp
0,D,16,1.726412e+12
1,D,73,1.726412e+12
2,U,16,1.726412e+12
3,U,73,1.726412e+12
4,D,32,1.726412e+12


In [9]:
def keywise_feature_generator():
    features = []
    last_down_idx = {}
    def next_feature(event_type: str, key: int, timestamp: float):
        nonlocal features
        if event_type == 'D':
            last_down_idx[key] = len(features)
            features.append(
                {
                    'key': key,
                    'down': timestamp,
                    'up': 0.0,
                }
            )
            yield {}
        else:
            features[last_down_idx[key]]['up'] = timestamp
            yield features[last_down_idx[key]]
    return next_feature

In [10]:
next_feature = keywise_feature_generator()
events = [
    ('D', 1, 0.1),
    ('D', 2, 0.2),
    ('U', 1, 0.25),
    ('U', 2, 0.3),
]

for event in events:
    print(next(next_feature(*event)))
# print(next(next_feature('D', 1, 0.1)))
# print(next(next_feature('D', 2, 0.2)))
# print(next(next_feature('U', 1, 0.25)))
# print(next(next_feature('U', 2, 0.3)))

{}
{}
{'key': 1, 'down': 0.1, 'up': 0.25}
{'key': 2, 'down': 0.2, 'up': 0.3}


In [11]:
df_feature_generator = keywise_feature_generator()
features = []
for event in df.itertuples():
    f = next(df_feature_generator(event.event_type, event.key, event.timestamp))
    if len(f) == 0:
        continue
    features.append(f)
    
features

[{'key': 16, 'down': 1726412251692.3606, 'up': 1726412251892.5317},
 {'key': 73, 'down': 1726412251810.3462, 'up': 1726412251892.5317},
 {'key': 32, 'down': 1726412252002.7615, 'up': 1726412252082.5757},
 {'key': 84, 'down': 1726412252144.7578, 'up': 1726412252216.1365},
 {'key': 82, 'down': 1726412252342.3472, 'up': 1726412252429.166},
 {'key': 73, 'down': 1726412252444.793, 'up': 1726412252533.5994},
 {'key': 69, 'down': 1726412252534.6526, 'up': 1726412252628.639},
 {'key': 68, 'down': 1726412252722.8936, 'up': 1726412252811.5208},
 {'key': 32, 'down': 1726412252779.7593, 'up': 1726412252865.494},
 {'key': 83, 'down': 1726412252973.0483, 'up': 1726412253061.2117},
 {'key': 79, 'down': 1726412253052.8442, 'up': 1726412253136.183},
 {'key': 32, 'down': 1726412253248.437, 'up': 1726412253345.579},
 {'key': 72, 'down': 1726412253458.7986, 'up': 1726412253532.637},
 {'key': 65, 'down': 1726412253532.637, 'up': 1726412253602.8596},
 {'key': 82, 'down': 1726412253682.9722, 'up': 1726412253

In [12]:
for i in range(1, len(features)):
    if features[i]['down'] < features[i-1]['down']:
        print('Not sorted at index', i)

In [13]:
# hold latency, inter-key latency, press latency, release latency, ascii
def create_feator_vector(feature, previous_feature=None):
    if previous_feature is None:
        return torch.tensor(
            [feature['up'] - feature['down'],
             0.0,
             0.0,
             0.0,
             feature['key'] / 128] 
        )
    
    return torch.tensor(
        [feature['up'] - feature['down'],
         feature['down'] - previous_feature['up'],
         feature['down'] - previous_feature['down'],
         feature['up'] - previous_feature['up'],
         feature['key'] / 128]
    )

def model_input_generator():
    previous_feature = None
    def next_input(feature):
        nonlocal previous_feature
        vec = create_feator_vector(feature, previous_feature)
        previous_feature = feature
        yield vec
    
    return next_input

In [14]:
model_input_gen = model_input_generator()
inputs = []
for feature in features:
    inputs.append(next(model_input_gen(feature)))
    print(inputs[-1])

tensor([2.0017e+02, 0.0000e+00, 0.0000e+00, 0.0000e+00, 1.2500e-01])
tensor([ 82.1855, -82.1855, 117.9856,   0.0000,   0.5703])
tensor([ 79.8142, 110.2297, 192.4153, 190.0439,   0.2500])
tensor([ 71.3787,  62.1821, 141.9963, 133.5608,   0.6562])
tensor([ 86.8188, 126.2107, 197.5894, 213.0295,   0.6406])
tensor([ 88.8064,  15.6270, 102.4458, 104.4333,   0.5703])
tensor([93.9863,  1.0532, 89.8596, 95.0396,  0.5391])
tensor([ 88.6272,  94.2546, 188.2410, 182.8818,   0.5312])
tensor([ 85.7346, -31.7615,  56.8657,  53.9731,   0.2500])
tensor([ 88.1633, 107.5544, 193.2891, 195.7178,   0.6484])
tensor([83.3389, -8.3674, 79.7959, 74.9714,  0.6172])
tensor([ 97.1421, 112.2539, 195.5928, 209.3960,   0.2500])
tensor([ 73.8384, 113.2195, 210.3616, 187.0579,   0.5625])
tensor([70.2227,  0.0000, 73.8384, 70.2227,  0.5078])
tensor([ 82.6936,  80.1125, 150.3352, 162.8062,   0.6406])
tensor([ 96.9158, 127.1921, 209.8857, 224.1079,   0.5312])
tensor([120.4985,   0.0000,  96.9158, 120.4985,   0.2500])
te

In [15]:
transformer(torch.stack(inputs))

tensor([[0.8976, 0.1840, 0.2840,  ..., 0.7051, 0.0928, 0.0428],
        [0.4014, 0.1517, 0.1177,  ..., 0.3880, 0.1678, 0.0370],
        [0.9846, 0.1572, 0.0314,  ..., 0.3968, 0.1249, 0.0415],
        ...,
        [0.9311, 0.0307, 0.1120,  ..., 0.6309, 0.0721, 0.0897],
        [0.9532, 0.1425, 0.0157,  ..., 0.5976, 0.0958, 0.0508],
        [0.8715, 0.0818, 0.0376,  ..., 0.6656, 0.0625, 0.0405]],
       dtype=torch.float64, grad_fn=<SigmoidBackward0>)

## Comparing similarity of embedding by splitting features into 5 sets

In [16]:
print(f'Number of features {len(features)}')
size = len(features)
num_of_sessions = 5
sessions = []
for i in range(num_of_sessions-1):
    sessions.append(
        torch.stack(inputs[i*size//num_of_sessions:(i+1)*size//num_of_sessions])
    )
sessions.append(
    torch.stack(inputs[(num_of_sessions-1)*size//num_of_sessions:])
)
print(f'Number of sessions {len(sessions)}')
print(f'Session lengths {[len(session) for session in sessions]}')

Number of features 158
Number of sessions 5
Session lengths [31, 32, 31, 32, 32]


In [17]:
def mean_of_vectors(t: torch.Tensor) -> torch.Tensor:
    return t.mean(dim=0)
    

embeddings = [mean_of_vectors(transformer(session)) for session in sessions]
embeddings

[tensor([0.8856, 0.1088, 0.0719, 0.8187, 0.8513, 0.8022, 0.1066, 0.0540, 0.8278,
         0.2389, 0.0863, 0.3549, 0.8734, 0.0908, 0.0935, 0.2500, 0.2885, 0.1399,
         0.5329, 0.0522, 0.6949, 0.0784, 0.0711, 0.5644, 0.2466, 0.8569, 0.0201,
         0.9906, 0.0324, 0.1078, 0.8818, 0.8508, 0.9450, 0.1885, 0.6847, 0.0632,
         0.5387, 0.3846, 0.8774, 0.6200, 0.0884, 0.2582, 0.2077, 0.1699, 0.8287,
         0.7358, 0.0848, 0.7760, 0.1141, 0.4266, 0.9005, 0.8832, 0.8684, 0.2185,
         0.1548, 0.1112, 0.8779, 0.4991, 0.7867, 0.5737, 0.8630, 0.4846, 0.0771,
         0.0614], dtype=torch.float64, grad_fn=<MeanBackward1>),
 tensor([0.8741, 0.1073, 0.0689, 0.8233, 0.8611, 0.8079, 0.0945, 0.0475, 0.8218,
         0.2402, 0.0814, 0.3235, 0.8747, 0.0917, 0.1017, 0.2817, 0.3276, 0.1501,
         0.4816, 0.0754, 0.6855, 0.0771, 0.0767, 0.5152, 0.2321, 0.8605, 0.0401,
         0.9867, 0.0352, 0.1075, 0.8893, 0.8622, 0.9379, 0.1804, 0.7189, 0.0604,
         0.5227, 0.3307, 0.8798, 0.6140, 0.0

In [18]:
embeddings[0].shape

torch.Size([64])

In [19]:
# cosine similarity for pairwise embeddings
def cosine_similarity(a: torch.Tensor, b: torch.Tensor) -> torch.Tensor:
    return torch.nn.functional.cosine_similarity(a, b, 0)

for i in range(num_of_sessions):
    print('session', i, end=' ')
    for j in range(i+1, num_of_sessions):
        print(cosine_similarity(embeddings[i], embeddings[j]), end=' ')
    print()

session 0 tensor(0.9993, dtype=torch.float64, grad_fn=<SumBackward1>) tensor(0.9991, dtype=torch.float64, grad_fn=<SumBackward1>) tensor(0.9986, dtype=torch.float64, grad_fn=<SumBackward1>) tensor(0.9995, dtype=torch.float64, grad_fn=<SumBackward1>) 
session 1 tensor(0.9991, dtype=torch.float64, grad_fn=<SumBackward1>) tensor(0.9988, dtype=torch.float64, grad_fn=<SumBackward1>) tensor(0.9997, dtype=torch.float64, grad_fn=<SumBackward1>) 
session 2 tensor(0.9995, dtype=torch.float64, grad_fn=<SumBackward1>) tensor(0.9991, dtype=torch.float64, grad_fn=<SumBackward1>) 
session 3 tensor(0.9989, dtype=torch.float64, grad_fn=<SumBackward1>) 
session 4 


In [20]:
user_embeddings = mean_of_vectors(torch.stack(embeddings))
user_embeddings

tensor([0.8878, 0.1056, 0.0634, 0.8283, 0.8608, 0.7920, 0.0998, 0.0500, 0.8245,
        0.2321, 0.0838, 0.3251, 0.8679, 0.0923, 0.1002, 0.2850, 0.3191, 0.1453,
        0.5079, 0.0656, 0.7058, 0.0790, 0.0746, 0.5515, 0.2381, 0.8542, 0.0362,
        0.9871, 0.0318, 0.1096, 0.8842, 0.8664, 0.9423, 0.1923, 0.6979, 0.0639,
        0.5406, 0.3488, 0.8752, 0.6164, 0.0885, 0.2457, 0.1946, 0.1619, 0.8302,
        0.7814, 0.0917, 0.7491, 0.1137, 0.4829, 0.9027, 0.8888, 0.8655, 0.2227,
        0.1525, 0.1074, 0.8680, 0.4883, 0.7790, 0.5963, 0.8671, 0.4718, 0.0783,
        0.0664], dtype=torch.float64, grad_fn=<MeanBackward1>)

## User embedding creation fast

In [21]:
df = pd.read_csv('../keystrokes_4.csv')
df['timestamp'] = df['timestamp'].map(lambda x: x/1000_000)

keywise_feat_gen = keywise_feature_generator()
keywise_feat = []
for event in df.itertuples():
    f = next(keywise_feat_gen(event.event_type, event.key, event.timestamp))
    if len(f) == 0:
        continue
    keywise_feat.append(f)
    
model_input_gen = model_input_generator()
inputs = []
for feature in keywise_feat:
    inputs.append(next(model_input_gen(feature)))

print(f'Number of features {len(keywise_feat)}')
embeddings = mean_of_vectors(transformer(torch.stack(inputs)))
embeddings

Number of features 158


tensor([0.8843, 0.0933, 0.0624, 0.8321, 0.8642, 0.7888, 0.1030, 0.0469, 0.8288,
        0.2399, 0.0806, 0.3230, 0.8656, 0.0951, 0.1032, 0.2979, 0.3261, 0.1437,
        0.5142, 0.0627, 0.7115, 0.0769, 0.0758, 0.5417, 0.2477, 0.8492, 0.0382,
        0.9864, 0.0325, 0.1059, 0.8838, 0.8683, 0.9420, 0.1832, 0.6840, 0.0664,
        0.5531, 0.3484, 0.8793, 0.6279, 0.0855, 0.2414, 0.1890, 0.1616, 0.8373,
        0.7719, 0.0896, 0.7725, 0.1092, 0.4770, 0.9051, 0.8872, 0.8698, 0.2199,
        0.1641, 0.1068, 0.8623, 0.4761, 0.7752, 0.6001, 0.8630, 0.4777, 0.0828,
        0.0660], dtype=torch.float64, grad_fn=<MeanBackward1>)

In [22]:
df_1 = pd.read_csv('../keystrokes_5.csv')
df_1['timestamp'] = df_1['timestamp'].map(lambda x: x/1000_000)

keywise_feat_gen = keywise_feature_generator()
model_input_gen = model_input_generator()
inputs = []
for event in df_1.itertuples():
    f = next(keywise_feat_gen(event.event_type, event.key, event.timestamp))
    if len(f) == 0:
        continue
    inputs.append(next(model_input_gen(f)))
        
embeddings1 = mean_of_vectors(transformer(torch.stack(inputs)))
embeddings1

tensor([0.9089, 0.1018, 0.0394, 0.8267, 0.8333, 0.7174, 0.1051, 0.0483, 0.8351,
        0.2359, 0.0839, 0.2503, 0.8154, 0.1243, 0.1360, 0.3906, 0.3920, 0.1700,
        0.4031, 0.0911, 0.7515, 0.0854, 0.0820, 0.6605, 0.2118, 0.8584, 0.1520,
        0.9677, 0.0357, 0.1297, 0.8890, 0.9060, 0.9287, 0.2646, 0.6207, 0.0864,
        0.5803, 0.3206, 0.8624, 0.6029, 0.0999, 0.2435, 0.1119, 0.1628, 0.8341,
        0.8420, 0.1007, 0.6486, 0.0968, 0.7165, 0.8851, 0.8877, 0.8443, 0.2534,
        0.1593, 0.1254, 0.8213, 0.5856, 0.8256, 0.5604, 0.8731, 0.4660, 0.0915,
        0.0740], dtype=torch.float64, grad_fn=<MeanBackward1>)

In [23]:
# euclidean distance
def euclidean_distance(a: torch.Tensor, b: torch.Tensor) -> torch.Tensor:
    return torch.nn.functional.pairwise_distance(a, b, p=2)
euclidean_distance(embeddings, embeddings1)

tensor(0.4413, dtype=torch.float64, grad_fn=<NormBackward1>)

In [24]:
df_2 = pd.read_csv('../keystrokes_6.csv')
df_2['timestamp'] = df_2['timestamp'].map(lambda x: x/1000_000)

keywise_feat_gen = keywise_feature_generator()
model_input_gen = model_input_generator()
inputs = []
for event in df_2.itertuples():
    f = next(keywise_feat_gen(event.event_type, event.key, event.timestamp))
    if len(f) == 0:
        continue
    inputs.append(next(model_input_gen(f)))
        
embeddings2 = mean_of_vectors(transformer(torch.stack(inputs)))
embeddings2

tensor([0.8900, 0.0978, 0.0516, 0.8293, 0.8549, 0.7664, 0.1017, 0.0466, 0.8241,
        0.2315, 0.0854, 0.3035, 0.8493, 0.1099, 0.1136, 0.3267, 0.3359, 0.1525,
        0.4710, 0.0772, 0.7308, 0.0826, 0.0803, 0.5861, 0.2196, 0.8551, 0.0650,
        0.9820, 0.0324, 0.1170, 0.8897, 0.8792, 0.9377, 0.2181, 0.6740, 0.0729,
        0.5589, 0.3328, 0.8731, 0.6100, 0.0923, 0.2305, 0.1646, 0.1652, 0.8371,
        0.7932, 0.0944, 0.7352, 0.1058, 0.5479, 0.9041, 0.8862, 0.8649, 0.2340,
        0.1637, 0.1075, 0.8549, 0.5177, 0.7926, 0.5898, 0.8761, 0.4756, 0.0853,
        0.0697], dtype=torch.float64, grad_fn=<MeanBackward1>)

In [25]:
print(euclidean_distance(embeddings, embeddings2))
print(euclidean_distance(embeddings1, embeddings2))
print(euclidean_distance(embeddings, embeddings1))
torch.nn.functional.cosine_similarity(embeddings, embeddings2, 0)

tensor(0.1465, dtype=torch.float64, grad_fn=<NormBackward1>)
tensor(0.3035, dtype=torch.float64, grad_fn=<NormBackward1>)
tensor(0.4413, dtype=torch.float64, grad_fn=<NormBackward1>)


tensor(0.9995, dtype=torch.float64, grad_fn=<SumBackward1>)

In [26]:
df_3 = pd.read_csv('../keystrokes_7.csv')
df_3['timestamp'] = df_3['timestamp'].map(lambda x: x/1000_000)

keywise_feat_gen = keywise_feature_generator()
model_input_gen = model_input_generator()
inputs = []
for event in df_3.itertuples():
    f = next(keywise_feat_gen(event.event_type, event.key, event.timestamp))
    if len(f) == 0:
        continue
    inputs.append(next(model_input_gen(f)))
    
embeddings3 = mean_of_vectors(transformer(torch.stack(inputs)))

In [27]:
print(euclidean_distance(embeddings, embeddings3))
print(euclidean_distance(embeddings, embeddings2))

tensor(0.0803, dtype=torch.float64, grad_fn=<NormBackward1>)
tensor(0.1465, dtype=torch.float64, grad_fn=<NormBackward1>)


In [28]:
df_10 = pd.read_csv('../keystrokes_10.csv')
df_10['timestamp'] = df_10['timestamp'].map(lambda x: x/1000_000)

keywise_feat_gen = keywise_feature_generator()
model_input_gen = model_input_generator()
inputs = []
for event in df_10.itertuples():
    f = next(keywise_feat_gen(event.event_type, event.key, event.timestamp))
    if len(f) == 0:
        continue
    inputs.append(next(model_input_gen(f)))
    
embeddings10 = mean_of_vectors(transformer(torch.stack(inputs)))

In [29]:
df_11 = pd.read_csv('../keystrokes_11.csv')
df_11['timestamp'] = df_11['timestamp'].map(lambda x: x/1000_000)

keywise_feat_gen = keywise_feature_generator()
model_input_gen = model_input_generator()
inputs = []
for event in df_11.itertuples():
    f = next(keywise_feat_gen(event.event_type, event.key, event.timestamp))
    if len(f) == 0:
        continue
    inputs.append(next(model_input_gen(f)))
    
embeddings11 = mean_of_vectors(transformer(torch.stack(inputs)))

In [31]:
euclidean_distance(embeddings, embeddings10)

tensor(0.1398, dtype=torch.float64, grad_fn=<NormBackward1>)

In [32]:
euclidean_distance(embeddings, embeddings11)

tensor(0.0756, dtype=torch.float64, grad_fn=<NormBackward1>)